In [ ]:
import rasterio
import numpy as np
import os
print("import modules")

# paths
output_path = r"D:\ASOML\Rockies\features\ASOML_CON_windScour_30_albn83_adjusts_colorado.tif"
dem_path = r"D:\ASOML\Rockies\wind_scourProcessing\ASO_dem_30m_CO_albn83.tif"
wind_path = r"D:\ASOML\Rockies\wind_scourProcessing\ASO_windDir_30m_CO_albn83.tif"


def compute_winstral_sx_raster(dem, wind_dir_deg, cellsize, dmax=200, sector_width=30, angle_step=5):
    """
    Compute the Winstral Sx index with a per-pixel wind direction raster.

    Parameters:
        dem (np.ndarray): 2D array of elevation values
        wind_dir_deg (np.ndarray): 2D array of wind direction (degrees)
        cellsize (float): Pixel resolution (meters)
        dmax (float): Max distance to search (meters)
        sector_width (float): Width of the wind direction sector (degrees)
        angle_step (float): Angle increment within sector (degrees)

    Returns:
        sx (np.ndarray): 2D array of Sx index values
    """
    rows, cols = dem.shape
    max_pixels = int(dmax / cellsize)
    sx = np.full_like(dem, np.nan, dtype=np.float32)

    sector_angles = np.arange(-sector_width / 2, sector_width / 2 + angle_step, angle_step)

    for i in range(rows):
        for j in range(cols):
            if np.isnan(dem[i, j]) or np.isnan(wind_dir_deg[i, j]):
                continue

            local_wind_deg = wind_dir_deg[i, j]
            local_slopes = []

            for delta_deg in sector_angles:
                angle_deg = (local_wind_deg + delta_deg) % 360
                angle_rad = np.deg2rad(angle_deg)

                for d in range(1, max_pixels + 1):
                    dx = int(np.round(d * np.cos(angle_rad)))
                    dy = int(np.round(d * np.sin(angle_rad)))
                    ni, nj = i + dy, j + dx

                    if 0 <= ni < rows and 0 <= nj < cols and not np.isnan(dem[ni, nj]):
                        elev_diff = dem[ni, nj] - dem[i, j]
                        slope = elev_diff / (d * cellsize)
                        local_slopes.append(slope)

            if local_slopes:
                local_slopes = np.array(local_slopes)
                max_slope = np.nanmax(local_slopes)
                min_slope = np.nanmin(local_slopes)
                sx_val = np.arctan(min_slope) if -min_slope > max_slope else np.arctan(max_slope)
                sx[i, j] = sx_val

    return sx

# apply function
with rasterio.open(dem_path) as dem_src, rasterio.open(wind_path) as wind_src:
    dem = dem_src.read(1)
    wind_dir = wind_src.read(1)
    profile = dem_src.profile
    transform = dem_src.transform
    cellsize = transform[0]

# Run function
sx_result = compute_winstral_sx_raster(dem, wind_dir, cellsize)

# Save output raster
profile.update(dtype=rasterio.float32, nodata=np.nan)
with rasterio.open(output_path, 'w', **profile) as dst:
    dst.write(sx_result, 1)

print("Saved enhanced Winstral Sx raster with per-pixel wind direction.")


import modules


C:\Users\etyrr\AppData\Local\Temp\ipykernel_15476\2283929655.py:51: RuntimeWarning: overflow encountered in scalar subtract
  elev_diff = dem[ni, nj] - dem[i, j]
